#### 네이버 검색 결과 크롤링 - 수원맛집

In [1]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from urllib.parse import quote
from bs4 import BeautifulSoup

In [2]:
url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='+ quote('수원맛집')
res = requests.get(url)
soup = BeautifulSoup(res.text)

In [28]:
lis = soup.select('.bx._svp_item')


[<li class="bx _svp_item" data-cr-area="rvw*x" data-cr-gdid="90000003_0000000000000033EFC071E0" data-cr-rank="1"> <div class="total_wrap api_ani_send"> <div class="total_area"> <div class="total_info"> <div class="api_save_group _keep_wrap"> <a aria-pressed="false" class="btn_save _keep_trigger" data-url="https://blog.naver.com/sky_923/223065698784" href="#" onclick="tCR('a=rvw*x.kep&amp;r=1&amp;i=90000003_0000000000000033EFC071E0&amp;u=javascript'); return false;" role="button"><i class="spnew ico_save">문서 저장하기</i></a> <div class="api_ly_save _keep_save_layer"> <a class="spnew_af item item_save _keep_save" data-cr-off="a=rvw*x.kepoff&amp;r=1&amp;i=90000003_0000000000000033EFC071E0&amp;u=javascript" data-cr-on="a=rvw*x.kepon&amp;r=1&amp;i=90000003_0000000000000033EFC071E0&amp;u=javascript" href="#" role="button">Keep에 저장</a> <a class="spnew_af item item_quick" href="https://keep.naver.com/" onclick="return goOtherCR(this,'a=rvw*x.kephome&amp;r=1&amp;i=90000003_0000000000000033EFC071E0&

-Selenium을 이용해서 데이터 가져오기

In [4]:
from selenium import webdriver
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(url)

In [5]:
# View 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

body = driver.find_element(By.TAG_NAME,'body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [6]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)

110

In [7]:
driver.close()

- 날짜, 제목, 내용 크롤링

In [10]:
li = lis[0]
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2023.04.05.'

In [11]:
title =  li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

'일품이던 수원맛집베스트10'

In [13]:
content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
content[:200]

'하러 수원맛집베스트10에 속한다는 갈비집을 다녀왔어요. 동네 주민들 뿐만 아니라 멀리서도 많은 사람들이 찾아가는 곳이라는데 소문대로 고퀄리티의 식사를 즐길 수 있던 곳이었어요. 고반가든 서수원점 주소 : 경기...'

- 날짜

In [15]:
from datetime import datetime, timedelta

today = datetime.now()
today

datetime.datetime(2023, 4, 25, 12, 10, 57, 906702)

In [21]:
# 2일전
date = today - timedelta(2)
f'{date.year}.{date.month:02d}.{date.day}'

'2023.4.23'

In [22]:
# 23시간 전
date = today - timedelta(hours=23)
f'{date.year}.{date.month:02d}.{date.day}'

'2023.4.24'

In [23]:
# 45분전
date = today - timedelta(minutes=45)
f'{date.year}.{date.month:02d}.{date.day}'

'2023.4.25'

In [25]:
def conver_date(s):
    if len(s) == 11:
        return s[:-1]
    today = datetime.now()
    if '어제' in s:
        date = datetime.now()
    elif '일' in s:
        date = today - timedelta(int(s[0]))
    elif '시간' in s:
        date = today - timedelta(hours=int(s[:-4]))
    elif '분' in s:
        date = date = today - timedelta(minutes=int(s[:-3]))
    return date.strftime('%Y.%m.%d')

In [26]:
lines = []
for li in lis:
    date = li.select_one('.sub_time.sub_txt').get_text().strip()
    date = conver_date(date)
    title =  li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    lines.append({'날짜':date, '제목':title, '내용':content})

In [27]:
df = pd.DataFrame(lines)
df.tail()

,날짜,제목,내용
105,2023.04.14,수원 맛집 가성비와 분위기 좋은 미도인,서울에서 핫한 미도인의 맛을 수원에서도 느낄 수 있어서 좋았어요 ! 경기 수원시 팔...
106,2023.04.13,"수원 맛집 소담화로 소갈비살 추천, 주차 정보",고기도 진짜 맛도리에요 >_< 가족외식 장소로도 회식장소로도 추천드려욤 #수원소고기...
107,2023.04.24,수원 맛집 호매실 돈가스 맛집 이백장돈가스 수원호매실점,#이백장돈가스 #이백장돈가스수원호매실점 #호매실돈가스 맛집 #호매실돈까스맛집 #수원...
108,2023.04.23,(수원맛집) 영양 가득 보양식 백세삼계탕,곳으로 수원 나가는 초입인 이목동에 위치해 잇어 접근성이 참 좋기도 합니다 지인이 ...
109,2023.04.16,재방문 200% 최애닭강정 <아들닭강정> 수원 맛집 내돈내산 주차가능,다음에도 맷집이네 놀러오세요~ #수원닭강정맛집 #수원치킨맛집 #수원통닭맛집리스트 #...
